In [1]:
import os
import h5py
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
data_dir='Outputs/'
from itertools import islice
from scipy import interpolate
import pyproj
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


# make sure we're dealing with the most recent version of any code we're using
%load_ext autoreload
%autoreload 2
%matplotlib widget

Read the files using h5py and compare the HDF5 groups and datasets:

## 2. A look at ATL03 over Byrd Glacier

This part of the tutorial is possible thanks to a function provided by Tyler Sutterly \[stand up, Tyler\].

To get an idea of how this works, we'll read in an ATL03 granule over Byrd Glacier in East Antarctica.  We'll use a reader that we import from an external Python file (readers/read_HDF5_ATL03.py, and a helper function, /readers/get_ATL03_x_atc.py), and plot the results using Matplotlib.  ATL03 is a complicated product, and reading it takes quite a few lines of code.  Please feel free to look at Tyler's code, and ask him questions about it.  We won't cover this in detail during the tutorial.

Read the files using h5py and compare the HDF5 groups and datasets:

In [2]:
def moving_average(iterable, n):
    # create an iterable object from input argument
    it = iter(iterable)
    piece = list(islice(it, n))
    while piece:
        yield np.mean(piece), np.median(piece), np.std(piece)
        # yield gives back a generator, which needs to be iterated upon to get the info out.
        piece = list(islice(it, n))

In [8]:
# it = iter(h_ph)
# n=2
# piece = list(islice(it, n))
# print(piece)
# while piece:
#     piece = list(islice(it, n))
#     print(piece)
# it = iter('ABCDEFGH')   #Python iterators can be consumed only once.
# print(list(islice(it, 2)))
# print(list(islice(it, 2)))
# print(list(islice(it, 2)))

[300.5837, 302.74335]
[301.58597, 301.52002]
[303.35022, 302.14496]
[301.52008, 303.53528]
[311.7203, 311.53104]
[311.97568, 312.82797]
[317.33817, 312.52444]
[310.80624, 311.58826]
[311.59186, 311.92758]
[311.56378, 313.40677]
[313.45114, 309.48492]
[311.8145, 312.78284]
[310.59607, 311.15845]
[313.38965, 311.5063]
[312.98022, 312.26758]
[310.56256, 311.6042]
[313.5639, 311.95297]
[311.55835, 312.0656]
[313.70523, 313.78326]
[311.7983, 312.28488]
[313.4644, 315.1077]
[313.40836, 312.27493]
[312.63843, 316.3656]
[313.2175, 311.36475]
[313.12067, 312.2639]
[311.4656, 312.691]
[313.94046, 313.1268]
[314.15958, 312.56198]
[312.66766, 313.45584]
[312.68393, 309.3195]
[312.51004, 312.82224]
[313.72864, 313.7513]
[311.61475, 312.19916]
[311.7798, 312.88702]
[311.306, 312.59726]
[312.47916, 312.80017]
[312.39096, 315.07098]
[312.9886, 314.41138]
[314.1289, 312.77103]
[314.64484, 313.3158]
[314.88498, 313.85092]
[313.1807, 313.53534]
[313.41452, 314.47595]
[313.2676, 313.96695]
[313.06284, 315

In [9]:
# li = np.array([0,1,1,1,1,0,0,0,1,0,1])
# for ii,item in enumerate(zip(li,li[1:])):
#     if item == (0,1):
#         print(ii)
# fun = interpolate.interp2d(lon_avg,lat_avg,havg,kind='linear')
# h_avg6 = fun(lon_li,lat_li)

# plt.figure(10)
# plt.plot(lat_avg,havg,'r.');
# plt.plot(lat_li,h_avg6)

/srv/conda/lib/python3.6/site-packages/scipy/interpolate/_fitpack_impl.py:976: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=1,1 nx,ny=107,101 m=10387 fp=1596.452207 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))


FigureCanvasNbAgg()

In [ ]:
#from Fernando Paolo & Johan Nilsson's utilities
def transform_coord(proj1, proj2, x, y):
    """Transform coordinates from proj1 to proj2 (EPSG num)."""
    
    # Set full EPSG projection strings
    proj1 = pyproj.Proj("+init=EPSG:"+proj1)
    proj2 = pyproj.Proj("+init=EPSG:"+proj2)
    
    # Convert coordinates
    return pyproj.transform(proj1, proj2, x, y)



In [3]:
%matplotlib widget

ATL03_file=glob('/home/jovyan/crackup/Outputs/*ATL03*.h5')
ATL06_file=glob('/home/jovyan/crackup/Outputs/*ATL06*.h5')
beamNum = [1,3,5]

f = h5py.File(ATL03_file[0], 'r')  # keep it open
beam = [k for k in f.keys() if k.startswith('gt')]

f6 = h5py.File(ATL06_file[0], 'r')
beam6 = [k for k in f6.keys() if k.startswith('gt')]

# Inspect base groups quickly
# for group in f.keys() :
#     print(group)
#     if group.startswith('gt1l'):
#         for gg in f[group].keys():
#             if gg.startswith('heights'):
#                 for dset in f[group][gg].keys():
#                     print(dset)

lookfor = ['delta_time','h_li','h_li_sigma','latitude','longitude','segment_id','sigma_geo_h']

fig1, ax1 = plt.subplots(nrows=2, ncols=1, sharex=True,figsize=(10,8))
fig2, ax2 = plt.subplots(nrows=2, ncols=1, sharex=True,figsize=(10,8))

data = {}
data6 = {}

window = 20
for ii,p in enumerate(beamNum):
    print('beam',beam[p])
    data[beam[p]] = {}
    data[beam[p]]['heights'] = {}

    for key,val in f[beam[p]]['heights'].items():
        data[beam[p]]['heights'][key] = val[:]

    #-- 0=Land; 1=Ocean; 2=SeaIce; 3=LandIce; 4=InlandWater
    conf = data[beam[p]]['heights']['signal_conf_ph'][:,3]
    lat_ph = np.array(data[beam[p]]['heights']['lat_ph'][conf>=2])
    lon_ph = np.array(data[beam[p]]['heights']['lon_ph'][conf>=2])
    h_ph   = np.array(data[beam[p]]['heights']['h_ph'][conf>=2])
    t_ph   = np.array(data[beam[p]]['heights']['delta_time'][conf>=2])
    
    havg = np.array([])
    hmed = np.array([])
    hstd= np.array([])
    for stats in moving_average(h_ph,window):
        havg = np.append(havg,stats[0])
        hmed = np.append(hmed,stats[1])
        hstd = np.append(hstd,stats[2])

    lat_avg = np.array([])
    for stats in moving_average(lat_ph,window):
        lat_avg = np.append(lat_avg,stats[0])
    lon_avg = np.array([])
    for stats in moving_average(lon_ph,window):
        lon_avg = np.append(lon_avg,stats[0])
    x_avg, y_avg = transform_coord('4326', '3031', lon_avg, lat_avg)

    data6[beam6[p]] = {}
    data6[beam6[p]]['land_ice_segments'] = {}
    
    for key,val in f6[beam6[p]]['land_ice_segments'].items():
        if key in lookfor:
            data6[beam6[p]]['land_ice_segments'][key] = val[:]
    h_li = data6[beam6[p]]['land_ice_segments']['h_li']
    h_li[h_li>3e38]= np.nan
    lat_li = data6[beam6[p]]['land_ice_segments']['latitude']
    lon_li = data6[beam6[p]]['land_ice_segments']['longitude']
    print(len(lon_li))
    pointx, pointy = transform_coord('4326', '3031', lon_li, lat_li)

    # print(find_nearest(pointlat,photlat[200]))
    elev_diff = np.zeros_like(y_avg)
    for i,reflat in enumerate(y_avg):
        dist_array = np.sqrt((pointx-x_avg[i])**2 + (pointy-reflat)**2)
        idx = np.where(dist_array == dist_array.min())
    #     idx = find_nearest(pointlat,reflat)
    #     if np.abs(pointlat[idx]-photlat[i]) <= 20:
        if dist_array[idx] <= 20:
            elev_diff[i] = pointh[idx] - h_ph[i]
        else:
            elev_diff[i] = np.NaN
    #         photlat[i] = 0
    
    if ii<2:
        ax1[ii].plot(lat_ph,h_ph,'.',color=(0.3,0.3,0.3),markersize=0.7,label='ATL03')
        ax1[ii].plot(lat_avg,havg,'r',linewidth=0.5,label='moving average')
        ax1[ii].plot(lat_avg,hmed,'g',linewidth=0.3,label='moving median')
        ax1[ii].plot(lat_avg,havg+hstd,'c--',linewidth=0.1,label='moving stdev')
        ax1[ii].plot(lat_avg,havg-hstd,'c--',linewidth=0.1)
        ax1[ii].plot(lat_li,h_li,'k.',label='ATL06')
        ax1[ii].set_ylim(0,80)
        ax1[ii].set_xlim(-64.38,-64.37)

        ax2[ii].plot(lat_avg,elev_diff,'b.',markersize=0.7,label='ATL06 minus ATL03')
        ax2[ii].set_ylim(-10,60)
        ax2[ii].set_xlim(-64.38,-64.37)

lgd = ax1[0].legend(loc=0,frameon=False)
lgd.get_frame().set_alpha(1.0)
for line in lgd.get_lines():
    line.set_linewidth(6)

lgd = ax2[0].legend(loc=2,frameon=False)
lgd.get_frame().set_alpha(1.0)
for line in lgd.get_lines():
    line.set_linewidth(6)


plt.show()
f.close()
f6.close()


FigureCanvasNbAgg()

FigureCanvasNbAgg()

beam gt1r
698
beam gt2r
700
beam gt3r
742
